## Allowing language models to choose the right algorithm
In this notebook we enable a language model to choose for the right algorithm. We define multiple segmentation algorithms / tools and then give the language model the choice which one to use given different inputs.

In [1]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.tools import tool

from skimage.io import imread
from napari_segment_blobs_and_things_with_membranes import voronoi_otsu_labeling, local_minima_seeded_watershed

import stackview

ModuleNotFoundError: No module named 'napari_segment_blobs_and_things_with_membranes'

Again, we define an image storage and a list of tools.

In [ ]:
image_storage = {}
tools = []

In [ ]:
@tools.append
@tool
def load_image(filename:str):
    """Useful for loading and image file and storing it."""
    print("loading", filename)
    image = imread(filename)
    image_storage[filename] = image
    return "The image is now stored as " + filename

We define two segmentation algorithms, one for segmenting bright objects and one for segmenting dark objects.

In [ ]:
@tools.append
@tool
def segment_bright_objects(image_name):
    """
    Useful for segmenting bright objects in an image 
    that has been loaded and stored before.
    """
    print("segmenting (Voronoi-Otsu-Labeling)", image_name)
    
    image = image_storage[image_name]
    label_image = voronoi_otsu_labeling(image, spot_sigma=4)
    
    label_image_name = "segmented_" + image_name
    image_storage[label_image_name] = label_image
    
    return "The segmented image has been stored as " + label_image_name

In [ ]:
@tools.append
@tool
def segment_dark_objects(image_name):
    """
    Useful for segmenting dark objects with bright border in an image 
    that has been loaded and stored before.
    """
    print("segmenting (Local-minima-seeded watershed)", image_name)
    
    image = image_storage[image_name]
    label_image = local_minima_seeded_watershed(image, spot_sigma=10)
    
    label_image_name = "segmented_" + image_name
    image_storage[label_image_name] = label_image
    
    return "The segmented image has been stored as " + label_image_name

In [ ]:
@tools.append
@tool
def show_image(image_name):
    """Useful for showing an image that has been loaded and stored before."""
    print("showing", image_name)
    
    image = image_storage[image_name]
    display(stackview.insight(image))
    
    return "The image " + image_name + " is shown above."

We create some memory and a large language model based on OpenAI's chatGPT.

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm=ChatOpenAI(temperature=0)

Given the list of tools, the large language model and the memory, we can create an agent.

In [ ]:
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
    memory=memory
)

This agent can then respond to prompts.

In [ ]:
agent.run("Please load the image ../../data/membrane2d.tif and show it.")

In [ ]:
agent.run("Please segment the image ../../data/membrane2d.tif")

In [ ]:
agent.run("Please show the segmented ../../data/membrane2d.tif image.")

The segmentation does not look like a cell-segmentation. Thus, we should ask more specifically.

In [ ]:
agent.run("Please segment the image ../../data/membrane2d.tif again and this time, segment the dark cells surrounded by bright membranes. Also show the result of the segmentation.")

We can also ask the agent which algorithm it chose and why it chose this tool.

In [ ]:
agent.run("Which algorithm did you use this time?")

In [ ]:
agent.run("Why did you use this algorithm?")

Note: The language model cannot see the image. Its tool selection depends on information you provided and information it acquired during the chat.